In [ ]:
# !pip3 install scikit-learn==1.6.0 pandas numpy matplotlib seaborn xgboost lime tdqm imblearn shap lime

In [ ]:
#=====================================================================
# IMPORTAÇÃO DE BIBLIOTECAS
# =====================================================================

# Manipulação e análise de dados
import pandas as pd
import numpy as np
import os
import pickle
import time
import logging
from datetime import datetime

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB

# Pré-processamento e splitting
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# Métricas de avaliação
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, confusion_matrix,
    f1_score, precision_score, recall_score, roc_auc_score, roc_curve, auc,
    precision_recall_curve, average_precision_score, classification_report
)

# Modelos clássicos
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Modelos avançados (boosting)
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Redes neurais (Keras/TensorFlow)
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import (
    Input, Activation, PReLU, LeakyReLU, Dense, Dropout,
    Conv1D, MaxPooling1D, Flatten, BatchNormalization,
    LSTM, Bidirectional, GlobalAveragePooling1D
)
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,
    TensorBoard, CSVLogger
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Balanceamento de dados
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from imblearn.combine import SMOTETomek

from tqdm import tqdm

# Interpretabilidade
import shap
from lime import lime_tabular

In [ ]:
# =====================================================================
# CONFIGURAÇÕES GERAIS
# =====================================================================
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("diabetes_prediction.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

np.random.seed(42)
tf.random.set_seed(42)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Criar diretório para salvar resultados
RESULTS_DIR = "resultados_diabetes"
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/modelos", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/graficos/confusao", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/graficos/roc", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/graficos/importancia", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/graficos/distribuicao", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/graficos/shap", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/graficos/lime", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/logs", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/history", exist_ok=True)

In [ ]:
# =====================================================================
# FUNÇÕES DE CARREGAMENTO E PRÉ-PROCESSAMENTO DE DADOS
# =====================================================================

# Dicionário de mapeamento das colunas para português
MAPEAMENTO_COLUNAS_PT = {
    'gender': 'gênero',
    'age': 'idade',
    'hypertension': 'hipertensão',
    'heart_disease': 'doença cardíaca',
    'smoking_history': 'histórico de tabagismo',
    'bmi': 'IMC',
    'HbA1c_level': 'nível de HbA1c',
    'blood_glucose_level': 'nível de glicose no sangue',
    'diabetes': 'diabetes'
}

# Função para renomear as colunas do DataFrame para português
def renomear_colunas_para_portugues(df):
    return df.rename(columns=MAPEAMENTO_COLUNAS_PT)

def carregar_dados(caminho_arquivo, verbose=True):
    """
    Carrega o dataset de predição de diabete e realiza análise exploratória inicial.

    Args:
        caminho_arquivo (str): Caminho para o arquivo CSV do dataset
        verbose (bool): Se True, exibe informações sobre o dataset

    Returns:
        pd.DataFrame: DataFrame com os dados carregados
    """
    logger.info(f"Carregando dados de {caminho_arquivo}")

    try:
        dataframe = pd.read_csv(caminho_arquivo)
        dataframe = renomear_colunas_para_portugues(dataframe)
        if verbose:
            logger.info(f"Dataset carregado com sucesso. Formato: {dataframe.shape}")
            logger.info(f"Colunas: {dataframe.columns.tolist()}")
            logger.info(f"Tipos de dados:\n{dataframe.dtypes}")
            logger.info(
                f"Distribuição da variável alvo (diabetes):\n{dataframe['diabetes'].value_counts(normalize=True) * 100}")

        return dataframe
    except Exception as e:
        logger.error(f"Erro ao carregar o dataset: {e}")


def analisar_dados(df):
    """
    Realiza análise exploratória detalhada dos dados.

    Args:
        df (pd.DataFrame): DataFrame com os dados

    Returns:
        dict: Dicionário com estatísticas e informações da análise
    """
    logger.info("Realizando análise exploratória dos dados")

    # Estatísticas básicas
    estatisticas = {
        'shape': df.shape,
        'missing_values': df.isnull().sum().to_dict(),
        'dtypes': df.dtypes.to_dict(),
        'target_distribution': df['diabetes'].value_counts(normalize=True).to_dict(),
        'numeric_stats': df.describe().to_dict(),
    }

    # Análise de variáveis categóricas
    cat_cols = df.select_dtypes(include=['object']).columns.tolist()
    estatisticas['categorical_counts'] = {col: df[col].value_counts().to_dict() for col in cat_cols}

    # Deteção de outliers (usando IQR)
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    outliers = {}
    for col in num_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[col] = {
            'lower_bound': lower_bound,
            'upper_bound': upper_bound,
            'n_outliers': ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
        }
    estatisticas['outliers'] = outliers

    # Correlações
    # Garante que apenas colunas numéricas sejam usadas para correlação
    numeric_df = df.select_dtypes(include=np.number)
    if 'diabetes' in numeric_df.columns:
        estatisticas['correlations'] = numeric_df.corr()['diabetes'].to_dict()
    else:
        estatisticas['correlations'] = {}
        logger.warning("Coluna 'diabetes' não encontrada ou não é numérica para cálculo de correlação.")

    # Visualizações
    visualizar_analise_exploratoria(df)

    return estatisticas


def visualizar_analise_exploratoria(df):
    """
    Cria visualizações para análise exploratória dos dados.

    Args:
        dataframe (pd.DataFrame): DataFrame com os dados
    """
    logger.info("Gerando visualizações para análise exploratória")
    # Configuração para visualizações
    plt.style.use('seaborn-v0_8-whitegrid')
    sns.set_palette('viridis')

    # 1. Distribuição da variável alvo
    plt.figure(figsize=(10, 6))
    ax = sns.countplot(x='diabetes', data=df, palette=['#3498db', '#e74c3c'], hue='diabetes', legend=False)
    plt.title('Distribuição da Variável Alvo (Diabetes)', fontsize=15)
    plt.xlabel('Diabetes', fontsize=12)
    plt.ylabel('Contagem', fontsize=12)

    # Adicionar percentagens
    total = len(df)
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x() + p.get_width() / 2., height + 5,
                f'{height} ({height / total:.1%})',
                ha="center", fontsize=12)

    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/graficos/distribuicao/distribuicao_target.png", dpi=300)
    plt.close()

    # 2. Distribuição das variáveis numéricas por status de diabete
    num_cols = ['idade', 'IMC', 'nível de HbA1c', 'nível de glicose no sangue']
    plt.figure(figsize=(15, 10))

    for i, col in enumerate(num_cols):
        plt.subplot(2, 2, i + 1)
        sns.histplot(data=df, x=col, hue='diabetes', kde=True, bins=30,
                     palette=['#3498db', '#e74c3c'], alpha=0.6)
        plt.title(f'Distribuição de {col} por Status de Diabetes', fontsize=13)
        plt.xlabel(col, fontsize=11)
        plt.ylabel('Contagem', fontsize=11)

    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/graficos/distribuicao/distribuicao_variaveis_numericas.png", dpi=300)
    plt.close()

    # 3. Boxplots para variáveis numéricas
    plt.figure(figsize=(15, 10))

    for i, col in enumerate(num_cols):
        plt.subplot(2, 2, i + 1)
        sns.boxplot(x='diabetes', y=col, data=df, palette=['#3498db', '#e74c3c'], hue='diabetes', legend=False)
        plt.title(f'{col} por Status de Diabetes', fontsize=13)
        plt.xlabel('Diabetes', fontsize=11)
        plt.ylabel(col, fontsize=11)

    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/graficos/boxplots_variaveis_numericas.png", dpi=300)
    plt.close()

    # 4. Matriz de correlação
    plt.figure(figsize=(12, 10))
    # Seleciona apenas colunas numéricas para a matriz de correlação
    num_df = df.select_dtypes(include=[np.number])
    corr_matrix = num_df.corr()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm',
                square=True, linewidths=0.5)
    plt.title('Matriz de Correlação', fontsize=15)
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/graficos/matriz_correlacao.png", dpi=300)
    plt.close()

    # 5. Pairplot para variáveis numéricas
    # Garante que só as colunas numéricas e a coluna alvo sejam usadas
    pairplot_cols = [col for col in num_cols if col in num_df.columns] + ['diabetes']
    sns.pairplot(df[pairplot_cols], hue='diabetes',
                 palette=['#3498db', '#e74c3c'], diag_kind='kde')
    plt.suptitle('Pairplot de Variáveis Numéricas', y=1.02, fontsize=16)
    plt.savefig(f"{RESULTS_DIR}/graficos/pairplot_variaveis_numericas.png", dpi=300)
    plt.close()

    # 6. Contagem de variáveis categóricas
    cat_cols = df.select_dtypes(include=['object']).columns.tolist()

    if cat_cols:
        plt.figure(figsize=(15, 5 * len(cat_cols)))

        for i, col in enumerate(cat_cols):
            plt.subplot(len(cat_cols), 1, i + 1)
            sns.countplot(x=col, hue='diabetes', data=df, palette=['#3498db', '#e74c3c'], legend=False)
            plt.title(f'Distribuição de {col} por Status de Diabetes', fontsize=13)
            plt.xlabel(col, fontsize=11)
            plt.ylabel('Contagem', fontsize=11)
            plt.xticks(rotation=45)

        plt.tight_layout()
        plt.savefig(f"{RESULTS_DIR}/graficos/distribuicao/distribuicao_variaveis_categoricas.png", dpi=300)
        plt.close()

    # 7. Relação entre HbA1c e glicose com diabete
    plt.figure(figsize=(12, 10))
    scatter = sns.scatterplot(data=df, x='nível de HbA1c', y='nível de glicose no sangue',
                              hue='diabetes', palette=['#3498db', '#e74c3c'],
                              s=80, alpha=0.7)
    plt.axvline(x=6.5, color='red', linestyle='--', label='Limiar HbA1c (6.5%)')
    plt.axhline(y=126, color='green', linestyle='--', label='Limiar Glicose (126 mg/dL)')

    # Adicionar anotações para os quadrantes
    plt.text(7.5, 200, 'Alto risco\n(HbA1c alto, Glicose alta)', fontsize=12, ha='center')
    plt.text(5.5, 200, 'Risco moderado\n(HbA1c normal, Glicose alta)', fontsize=12, ha='center')
    plt.text(7.5, 100, 'Risco moderado\n(HbA1c alto, Glicose normal)', fontsize=12, ha='center')
    plt.text(5.5, 100, 'Baixo risco\n(HbA1c normal, Glicose normal)', fontsize=12, ha='center')

    plt.title('Relação entre HbA1c e Glicose no Sangue', fontsize=15)
    plt.xlabel('Nível de HbA1c (%)', fontsize=12)
    plt.ylabel('Nível de Glicose (mg/dL)', fontsize=12)
    plt.legend(loc='upper left')
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/graficos/relacao_hba1c_glicose.png", dpi=300)
    plt.close()


def pre_processar_dados(dataframe, metodo_normalizacao='standard',
                        metodo_balanceamento='smote',
                        tratar_outliers=True,
                        metodo_outliers='iqr',
                        fator_outliers=1.5,
                        percentis_outliers=(5, 95),
                        test_size=0.2,
                        val_size=0.15,
                        random_state=42):
    """
    Realiza o pré-processamento completo dos dados para treinamento do modelo.

    Args:
        dataframe (pd.DataFrame): DataFrame com os dados
        metodo_normalizacao (str): Método de normalização ('standard', 'minmax', 'robust')
        metodo_balanceamento (str): Método de balanceamento ('smote', 'adasyn', 'borderline', 'tomek', 'none')
        tratar_outliers (bool): Se True, trata outliers
        metodo_outliers (str): Método para tratar outliers ('iqr', 'zscore', 'percentil', 'isolation_forest', 'winsorization')
        fator_outliers (float): Fator multiplicativo para IQR ou threshold para Z-score
        percentis_outliers (tuple): Percentis inferior e superior para método percentil
        test_size (float): Proporção do conjunto de teste
        val_size (float): Proporção do conjunto de validação (em relação ao conjunto não-teste)
        random_state (int): Semente aleatória

    Returns:
        tuple: (x_train, x_val, x_test, y_train, y_val, y_test, preprocessador, feature_names)
    """
    logger.info("Iniciando pré-processamento dos dados")

    # Remover valores 'Other' da coluna gender (se existir)
    if 'gender' in dataframe.columns and 'Other' in dataframe['gender'].unique():
        logger.info("Removendo valores 'Other' da coluna gender")
        dataframe = dataframe[dataframe['gender'] != 'Other']

    # Separação entre atributos e rótulo
    x = dataframe.drop(columns=['diabetes']).copy()
    y = dataframe['diabetes'].copy()

    # Identificar colunas numéricas e categóricas
    colunas_numericas = x.select_dtypes(include=['int64', 'float64']).columns.tolist()
    colunas_categoricas = x.select_dtypes(include=['object']).columns.tolist()

    logger.info(f"Colunas numéricas: {colunas_numericas}")
    logger.info(f"Colunas categóricas: {colunas_categoricas}")

    # Armazenar nomes das features antes da codificação
    feature_names_original = x.columns.tolist()

    # Codificar variáveis categóricas
    encoder = None
    if colunas_categoricas:
        logger.info("Codificando variáveis categóricas usando One-Hot Encoding")
        encoder = OneHotEncoder(drop='first', sparse_output=False)

        # Aplicar one-hot encoding
        categorical_encoded = encoder.fit_transform(x[colunas_categoricas])

        # Obter nomes das novas colunas
        categorical_feature_names = encoder.get_feature_names_out(colunas_categoricas)

        # Criar DataFrame com dados codificados
        categorical_df = pd.DataFrame(categorical_encoded,
                                    columns=categorical_feature_names,
                                    index=x.index)

        # Combinar dados numéricos com categóricos codificados
        x_processed = pd.concat([x[colunas_numericas], categorical_df], axis=1)

        # Atualizar lista de nomes das features
        feature_names = colunas_numericas + categorical_feature_names.tolist()

        logger.info(f"Variáveis categóricas codificadas. Novas features: {categorical_feature_names.tolist()}")
    else:
        x_processed = x.copy()
        feature_names = feature_names_original

    # Atualizar lista de colunas numéricas (agora inclui todas as colunas)
    colunas_numericas_final = x_processed.columns.tolist()

    # Tratamento de outliers
    if tratar_outliers:
        logger.info(f"Tratando outliers usando método '{metodo_outliers}'")
        x_processed = tratar_outliers_avancado(
            x_processed, colunas_numericas_final,
            metodo=metodo_outliers,
            fator=fator_outliers,
            percentis=percentis_outliers
        )

    # Normalização
    scaler = None
    if metodo_normalizacao == 'standard':
        logger.info("Aplicando normalização padrão (StandardScaler)")
        scaler = StandardScaler()
        x_processed[colunas_numericas_final] = scaler.fit_transform(x_processed[colunas_numericas_final])
    elif metodo_normalizacao == 'minmax':
        logger.info("Aplicando normalização Min-Max (MinMaxScaler)")
        scaler = MinMaxScaler()
        x_processed[colunas_numericas_final] = scaler.fit_transform(x_processed[colunas_numericas_final])
    elif metodo_normalizacao == 'robust':
        logger.info("Aplicando normalização robusta (RobustScaler)")
        scaler = RobustScaler()
        x_processed[colunas_numericas_final] = scaler.fit_transform(x_processed[colunas_numericas_final])
    else:
        logger.warning(f"Método de normalização '{metodo_normalizacao}' não reconhecido. Usando dados originais.")

    # Converter para numpy array para algoritmos de balanceamento
    x_array = x_processed.values
    y_array = y.values

    # Balanceamento de classes
    if metodo_balanceamento == 'smote':
        logger.info("Aplicando SMOTE para balanceamento de classes")
        smote = SMOTE(random_state=random_state)
        x_array, y_array = smote.fit_resample(x_array, y_array)
    elif metodo_balanceamento == 'adasyn':
        logger.info("Aplicando ADASYN para balanceamento de classes")
        adasyn = ADASYN(random_state=random_state)
        x_array, y_array = adasyn.fit_resample(x_array, y_array)
    elif metodo_balanceamento == 'borderline':
        logger.info("Aplicando BorderlineSMOTE para balanceamento de classes")
        borderline = BorderlineSMOTE(random_state=random_state)
        x_array, y_array = borderline.fit_resample(x_array, y_array)
    elif metodo_balanceamento == 'tomek':
        logger.info("Aplicando SMOTETomek para balanceamento de classes")
        tomek = SMOTETomek(random_state=random_state)
        x_array, y_array = tomek.fit_resample(x_array, y_array)
    elif metodo_balanceamento == 'none':
        logger.info("Nenhum balanceamento aplicado")
    else:
        logger.warning(f"Método de balanceamento '{metodo_balanceamento}' não reconhecido. Usando dados originais.")

    logger.info(f"Dados após balanceamento: {x_array.shape}, distribuição das classes: {np.bincount(y_array.astype(int))}")

    # Divisão em conjuntos de treino, validação e teste
    x_train, x_temp, y_train, y_temp = train_test_split(
        x_array, y_array, test_size=test_size, random_state=random_state, stratify=y_array
    )
    x_val, x_test, y_val, y_test = train_test_split(
        x_temp, y_temp, test_size=val_size / (test_size), random_state=random_state, stratify=y_temp
    )

    # Salvar pré-processador
    preprocessador = {
        'scaler': scaler,
        'encoder': encoder,
        'feature_selection': None,  # Placeholder para futura seleção de features
        'feature_names_original': feature_names_original,
        'feature_names_final': feature_names
    }

    logger.info(f"Divisão dos dados concluída:")
    logger.info(f"  Treino: {x_train.shape}")
    logger.info(f"  Validação: {x_val.shape}")
    logger.info(f"  Teste: {x_test.shape}")

    return x_train, x_val, x_test, y_train, y_val, y_test, preprocessador, feature_names


def tratar_outliers_avancado(df, colunas_numericas, metodo='iqr', fator=1.5, percentis=(5, 95)):
    """
    Trata outliers de forma mais eficiente usando diferentes métodos.

    Args:
        df (pd.DataFrame): DataFrame com os dados
        colunas_numericas (list): Lista de colunas numéricas
        metodo (str): Método para tratar outliers ('iqr', 'zscore', 'percentil', 'isolation_forest')
        fator (float): Fator multiplicativo para IQR ou threshold para Z-score
        percentis (tuple): Percentis inferior e superior para método percentil

    Returns:
        pd.DataFrame: DataFrame com outliers tratados
    """
    df_tratado = df.copy()

    if metodo == 'iqr':
        logger.info(f"Tratando outliers usando IQR com fator {fator}")
        # Calcular todos os quantis de uma vez (operação vetorizada)
        q1 = df_tratado[colunas_numericas].quantile(0.25)
        q3 = df_tratado[colunas_numericas].quantile(0.75)
        iqr = q3 - q1
        lower_bounds = q1 - fator * iqr
        upper_bounds = q3 + fator * iqr

        # Aplicar capping vetorizado
        df_tratado[colunas_numericas] = df_tratado[colunas_numericas].clip(
            lower=lower_bounds, upper=upper_bounds, axis=1
        )

    elif metodo == 'zscore':
        logger.info(f"Tratando outliers usando Z-score com threshold {fator}")
        from scipy import stats

        # Calcular Z-scores para todas as colunas
        z_scores = np.abs(stats.zscore(df_tratado[colunas_numericas]))

        # Criar máscara para outliers
        outlier_mask = (z_scores > fator).any(axis=1)
        logger.info(f"Identificados {outlier_mask.sum()} outliers com Z-score > {fator}")

        # Substituir outliers pela mediana da coluna
        for col in colunas_numericas:
            col_z_scores = np.abs(stats.zscore(df_tratado[col]))
            outliers = col_z_scores > fator
            mediana = df_tratado[col].median()
            df_tratado.loc[outliers, col] = mediana

    elif metodo == 'percentil':
        logger.info(f"Tratando outliers usando percentis {percentis}")
        # Calcular percentis para todas as colunas
        lower_bounds = df_tratado[colunas_numericas].quantile(percentis[0]/100)
        upper_bounds = df_tratado[colunas_numericas].quantile(percentis[1]/100)

        # Aplicar capping vetorizado
        df_tratado[colunas_numericas] = df_tratado[colunas_numericas].clip(
            lower=lower_bounds, upper=upper_bounds, axis=1
        )

    elif metodo == 'isolation_forest':
        logger.info("Tratando outliers usando Isolation Forest")
        from sklearn.ensemble import IsolationForest

        # Treinar Isolation Forest
        iso_forest = IsolationForest(contamination=0.1, random_state=42)
        outlier_pred = iso_forest.fit_predict(df_tratado[colunas_numericas])

        # Identificar outliers (-1 indica outlier)
        outliers = outlier_pred == -1
        logger.info(f"Identificados {outliers.sum()} outliers com Isolation Forest")

        # Substituir outliers pela mediana
        for col in colunas_numericas:
            mediana = df_tratado[col].median()
            df_tratado.loc[outliers, col] = mediana

    elif metodo == 'winsorization':
        logger.info(f"Tratando outliers usando Winsorização com percentis {percentis}")
        from scipy.stats.mstats import winsorize

        # Aplicar winsorização para cada coluna
        for col in colunas_numericas:
            df_tratado[col] = winsorize(
                df_tratado[col],
                limits=((100-percentis[1])/100, (100-percentis[0])/100)
            )

    else:
        raise ValueError(f"Método '{metodo}' não reconhecido. Use 'iqr', 'zscore', 'percentil', 'isolation_forest' ou 'winsorization'")

    # Calcular estatísticas de outliers removidos
    outliers_removidos = {}
    for col in colunas_numericas:
        valores_originais = df[col]
        valores_tratados = df_tratado[col]
        modificados = (valores_originais != valores_tratados).sum()
        outliers_removidos[col] = {
            'modificados': modificados,
            'percentual': (modificados / len(df)) * 100
        }

    logger.info("Resumo do tratamento de outliers:")
    for col, stats in outliers_removidos.items():
        logger.info(f"{col}: {stats['modificados']} valores modificados ({stats['percentual']:.2f}%)")

    return df_tratado

In [ ]:
# =====================================================================
# FUNÇÕES PARA CRIAÇÃO DE MODELOS
# =====================================================================

def criar_modelo_mlp(input_shape,
                     camadas=[128, 64, 32],
                     activations=['relu', 'relu', 'relu'],
                     dropout_rates=[0.3, 0.3, 0.3],
                     regularization=0.001,
                     learning_rate=0.005,
                     batch_norm=True):
    """
    Cria um modelo MLP (Perceptron Multicamadas) para classificação binária.

    Args:
        input_shape (tuple): Forma dos dados de entradas
        camadas (list): Lista com o número de neurônios em cada camada oculta
        activations (list): Lista com as funções de ativação para cada camada
        dropout_rates (list): Lista com as taxas de dropout para cada camada
        regularization (float): Coeficiente de regularização L2
        learning_rate (float): Taxa de aprendizado
        batch_norm (bool): Se True, adiciona camadas de Batch Normalization

    Returns:
        tf.keras.Model: Modelo MLP compilado
    """
    logger.info(f"Criando modelo MLP com {len(camadas)} camadas ocultas")
    logger.info(f"Arquitetura: {camadas}")
    logger.info(f"Ativações: {activations}")
    logger.info(f"Dropout rates: {dropout_rates}")

    # Verificar se os parâmetros têm o mesmo comprimento
    assert len(camadas) == len(activations) == len(dropout_rates), \
        "camadas, ativacoes e dropout_rates devem ter o mesmo comprimento"

    # Criar modelo
    model = Sequential()

    # Camada de entrada
    model.add(Input(shape=input_shape))

    # Camadas ocultas
    for i, (units, activation, dropout_rate) in enumerate(zip(camadas, activations, dropout_rates)):
        # Adicionar camada densa com regularização L2
        model.add(Dense(
            units=units,
            kernel_regularizer=l2(regularization),
            name=f'dense_{i + 1}'
        ))

        # Adicionar batch normalization antes da ativação
        if batch_norm:
            model.add(BatchNormalization(name=f'batch_norm_{i + 1}'))

        # Adicionar função de ativação
        if activation == 'leaky_relu':
            model.add(LeakyReLU(alpha=0.1, name=f'leaky_relu_{i + 1}'))
        elif activation == 'prelu':
            model.add(PReLU(name=f'prelu_{i + 1}'))
        else:
            model.add(Activation(activation, name=f'activation_{i + 1}'))

        # Adicionar dropout para regularização
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate, name=f'dropout_{i + 1}'))

    # Camada de saída
    model.add(Dense(1, activation='sigmoid', name='output'))

    # Compilar modelo
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            'recall'
        ]
    )

    # Resumo do modelo
    model.summary()

    return model


def criar_modelo_cnn(input_shape,
                     filtros=[64, 32, 16],
                     kernel_sizes=[3, 3, 3],
                     ativacoes=['relu', 'relu', 'relu'],
                     dropout_rates=[0.3, 0.3, 0.3],
                     pool_sizes=[2, 2, 2],
                     regularizacao=0.001,
                     learning_rate=0.001,
                     batch_norm=True):
    """
    Cria um modelo CNN 1D para classificação binária de dados tabulares.

    Args:
        input_shape (tuple): Forma dos dados de entrada (deve ser 3D para CNN)
        filtros (list): Lista com o número de filtros em cada camada convolucional
        kernel_sizes (list): Lista com os tamanhos do kernel para cada camada
        ativacoes (list): Lista com as funções de ativação para cada camada
        dropout_rates (list): Lista com as taxas de dropout para cada camada
        pool_sizes (list): Lista com os tamanhos de pooling para cada camada
        regularizacao (float): Coeficiente de regularização L2
        learning_rate (float): Taxa de aprendizado
        batch_norm (bool): Se True, adiciona camadas de Batch Normalization

    Returns:
        tf.keras.Model: Modelo CNN compilado
    """
    logger.info(f"Criando modelo CNN com {len(filtros)} camadas convolucionais")
    logger.info(f"Filtros: {filtros}")
    logger.info(f"Kernel sizes: {kernel_sizes}")
    logger.info(f"Ativações: {ativacoes}")

    # Verificar se os parâmetros têm o mesmo comprimento
    assert len(filtros) == len(kernel_sizes) == len(ativacoes) == len(dropout_rates) == len(pool_sizes), \
        "filtros, kernel_sizes, ativacoes, dropout_rates e pool_sizes devem ter o mesmo comprimento"

    # Verificar se a forma de entrada é 3D
    if len(input_shape) != 2:
        logger.warning(f"Input shape {input_shape} não é 3D. Adicionando dimensão extra.")
        input_shape = (*input_shape, 1)

    # Criar modelo
    model = Sequential()

    # Camada de entrada
    model.add(Input(shape=input_shape))

    # Camadas convolucionais
    for i, (filters, kernel_size, activation, dropout_rate, pool_size) in enumerate(
            zip(filtros, kernel_sizes, ativacoes, dropout_rates, pool_sizes)):

        # Adicionar camada convolucional com regularização L2
        model.add(Conv1D(
            filters=filters,
            kernel_size=kernel_size,
            padding='same',
            kernel_regularizer=l2(regularizacao),
            name=f'conv_{i + 1}'
        ))

        # Adicionar batch normalization antes da ativação
        if batch_norm:
            model.add(BatchNormalization(name=f'batch_norm_{i + 1}'))

        # Adicionar função de ativação
        if activation == 'leaky_relu':
            model.add(LeakyReLU(alpha=0.1, name=f'leaky_relu_{i + 1}'))
        elif activation == 'prelu':
            model.add(PReLU(name=f'prelu_{i + 1}'))
        else:
            model.add(Activation(activation, name=f'activation_{i + 1}'))

        # Adicionar max pooling
        model.add(MaxPooling1D(pool_size=pool_size, padding='same', name=f'pool_{i + 1}'))

        # Adicionar dropout para regularização
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate, name=f'dropout_{i + 1}'))

    # Flatten para conectar às camadas densas
    model.add(Flatten(name='flatten'))

    # Camada densa intermediária
    model.add(Dense(32, kernel_regularizer=l2(regularizacao), name='dense_1'))
    if batch_norm:
        model.add(BatchNormalization(name='batch_norm_dense'))
    model.add(Activation('relu', name='activation_dense'))
    model.add(Dropout(0.3, name='dropout_dense'))

    # Camada de saída
    model.add(Dense(1, activation='sigmoid', name='output'))

    # Compilar modelo
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            'recall'
        ]
    )

    # Resumo do modelo
    model.summary()

    return model


def criar_modelo_hibrido(input_shape,
                         filtros_cnn=[64, 32],
                         kernel_sizes=[3, 3],
                         unidades_lstm=32,
                         unidades_densas=[64, 32],
                         ativacoes=['relu', 'relu'],
                         dropout_rates=[0.3, 0.3],
                         regularizacao=0.001,
                         learning_rate=0.001,
                         batch_norm=True):
    """
    Cria um modelo híbrido CNN-LSTM para classificação binária de dados tabulares.

    Args:
        input_shape (tuple): Forma dos dados de entrada (deve ser 3D)
        filtros_cnn (list): Lista com o número de filtros em cada camada convolucional
        kernel_sizes (list): Lista com os tamanhos do kernel para cada camada
        unidades_lstm (int): Número de unidades na camada LSTM
        unidades_densas (list): Lista com o número de neurônios em cada camada densa
        ativacoes (list): Lista com as funções de ativação para cada camada densa
        dropout_rates (list): Lista com as taxas de dropout para cada camada densa
        regularizacao (float): Coeficiente de regularização L2
        learning_rate (float): Taxa de aprendizado
        batch_norm (bool): Se True, adiciona camadas de Batch Normalization

    Returns:
        tf.keras.Model: Modelo híbrido CNN-LSTM compilado
    """
    logger.info("Criando modelo híbrido CNN-LSTM")
    logger.info(f"Filtros CNN: {filtros_cnn}")
    logger.info(f"Unidades LSTM: {unidades_lstm}")
    logger.info(f"Unidades densas: {unidades_densas}")

    # Verificar se a forma de entrada é 3D
    if len(input_shape) != 2:
        logger.warning(f"Input shape {input_shape} não é 3D. Adicionando dimensão extra.")
        input_shape = (*input_shape, 1)

    # Verificar se os parâmetros têm o mesmo comprimento
    assert len(filtros_cnn) == len(kernel_sizes), \
        "filtros_cnn e kernel_sizes devem ter o mesmo comprimento"
    assert len(unidades_densas) == len(ativacoes) == len(dropout_rates), \
        "unidades_densas, ativacoes e dropout_rates devem ter o mesmo comprimento"

    # Criar modelo usando a API funcional
    inputs = Input(shape=input_shape)

    # Camadas convolucionais
    x = inputs
    for i, (filters, kernel_size) in enumerate(zip(filtros_cnn, kernel_sizes)):
        x = Conv1D(
            filters=filters,
            kernel_size=kernel_size,
            padding='same',
            kernel_regularizer=l2(regularizacao),
            name=f'conv_{i + 1}'
        )(x)

        if batch_norm:
            x = BatchNormalization(name=f'batch_norm_conv_{i + 1}')(x)

        x = Activation('relu', name=f'activation_conv_{i + 1}')(x)
        x = MaxPooling1D(pool_size=2, padding='same', name=f'pool_{i + 1}')(x)

    # Camada LSTM bidirecional
    x = Bidirectional(LSTM(unidades_lstm, return_sequences=True, name='lstm_1'))(x)
    x = Dropout(0.3, name='dropout_lstm')(x)

    # Global Average Pooling
    x = GlobalAveragePooling1D(name='global_avg_pool')(x)

    # Camadas densas
    for i, (units, activation, dropout_rate) in enumerate(zip(unidades_densas, ativacoes, dropout_rates)):
        x = Dense(units, kernel_regularizer=l2(regularizacao), name=f'dense_{i + 1}')(x)

        if batch_norm:
            x = BatchNormalization(name=f'batch_norm_dense_{i + 1}')(x)

        if activation == 'leaky_relu':
            x = LeakyReLU(alpha=0.1, name=f'leaky_relu_{i + 1}')(x)
        elif activation == 'prelu':
            x = PReLU(name=f'prelu_{i + 1}')(x)
        else:
            x = Activation(activation, name=f'activation_dense_{i + 1}')(x)

        if dropout_rate > 0:
            x = Dropout(dropout_rate, name=f'dropout_dense_{i + 1}')(x)

    # Camada de saída
    outputs = Dense(1, activation='sigmoid', name='output')(x)

    # Criar e compilar modelo
    model = Model(inputs=inputs, outputs=outputs)

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            'recall'
        ]
    )

    # Resumo do modelo
    model.summary()

    return model


def criar_callbacks(nome_modelo, paciencia=10, min_delta=0.001, fator_reducao=0.5, min_lr=1e-6):
    """
    Cria callbacks para treinamento do modelo.

    Args:
        nome_modelo (str): Nome do modelo para salvar
        paciencia (int): Número de épocas para esperar antes de parar o treinamento
        min_delta (float): Mínima mudança para considerar como melhoria
        fator_reducao (float): Fator para reduzir a taxa de aprendizado
        min_lr (float): Taxa de aprendizado mínima

    Returns:
        list: Lista de callbacks
    """
    # Criar diretório para ‘logs’ do TensorBoard
    log_dir = f"{RESULTS_DIR}/logs/{nome_modelo}_{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    os.makedirs(log_dir, exist_ok=True)

    # Callbacks
    callbacks = [
        # Early stopping para evitar overfitting
        EarlyStopping(
            monitor='val_loss',
            patience=paciencia,
            min_delta=min_delta,
            restore_best_weights=True,
            verbose=1
        ),

        # Model checkpoint para salvar o melhor modelo
        ModelCheckpoint(
            filepath=f"{RESULTS_DIR}/modelos/{nome_modelo}_best.h5",
            monitor='val_auc',
            mode='max',
            save_best_only=True,
            verbose=1
        ),

        # Reduce learning rate quando o treinamento estagnar
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=fator_reducao,
            patience=paciencia // 2,
            min_lr=min_lr,
            verbose=1
        ),

        # TensorBoard para visualização do treino
        TensorBoard(
            log_dir=log_dir,
            histogram_freq=1,
            write_graph=True,
            update_freq='epoch'
        ),

        # CSV Logger para salvar histórico de treino
        CSVLogger(
            filename=f"{RESULTS_DIR}/history/{nome_modelo}_history.csv",
            separator=',',
            append=False
        )
    ]

    return callbacks

In [ ]:
# =====================================================================
# FUNÇÕES PARA COMPARAÇÃO DE MODELOS
# =====================================================================

def treinar_modelos_classicos(x_train, y_train, x_test, y_test, feature_names):
    """
    Treina e avalia modelos clássicos de machine learning.

    Args:
        x_train (np.ndarray): Dados de treinamento
        y_train (np.ndarray): Rótulos de treinamento
        x_test (np.ndarray): Dados de teste
        y_test (np.ndarray): Rótulos de teste
        feature_names (list): Nomes das features

    Returns:
        pd.DataFrame: DataFrame com resultados dos modelos
    """
    logger.info("Treinando modelos clássicos de machine learning")

    # Garantir que x_train e x_test sejam DataFrames com nomes de features
    if not isinstance(x_train, pd.DataFrame):
        x_train = pd.DataFrame(x_train, columns=feature_names)
    if not isinstance(x_test, pd.DataFrame):
        x_test = pd.DataFrame(x_test, columns=feature_names)

    # Definir modelos
    modelos = {
        'Regressão Logística': LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced', verbose=1),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', verbose=1),
        'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42, verbose=1),
        'XGBoost': XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=42, verbose=1, n_jobs=-1),
        'LightGBM': LGBMClassifier(random_state=42, verbose=1, n_jobs=-1),
        'SVM': SVC(kernel='rbf', probability=True, random_state=42, class_weight='balanced', verbose=1),
        'KNN': KNeighborsClassifier(n_neighbors=5),
        'Decision Tree': DecisionTreeClassifier(random_state=42, class_weight='balanced'),
        'Naive Bayes': GaussianNB(),
    }

    # Resultados
    resultados = []

    # Treinar e avaliar cada modelo
    for nome, modelo in modelos.items():
        logger.info(f"Treinando modelo: {nome}")

        # Treinar modelo
        modelo.fit(x_train, y_train)

        # Fazer predições
        y_pred = modelo.predict(x_test)

        # Obter probabilidades
        if hasattr(modelo, "predict_proba"):
            y_prob = modelo.predict_proba(x_test)[:, 1]
        else:
            # Para SVM sem probabilidades
            y_prob = modelo.decision_function(x_test)
            # Normalizar para [0, 1]
            y_prob = (y_prob - y_prob.min()) / (y_prob.max() - y_prob.min())

        # Calcular métricas
        metrics = {
            'modelo': nome,
            'accuracy': accuracy_score(y_test, y_pred),
            'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_prob),
            'average_precision': average_precision_score(y_test, y_prob)
        }

        resultados.append(metrics)

        # Salvar modelo
        with open(f"{RESULTS_DIR}/modelos/modelo_{nome.replace(' ', '_').lower()}.pkl", 'wb') as f:
            pickle.dump(modelo, f)

        # Visualizar matriz de confusão
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
        plt.title(f'Matriz de Confusão - {nome}', fontsize=15)
        plt.ylabel('Valor Real', fontsize=12)
        plt.xlabel('Valor Predito', fontsize=12)
        plt.tight_layout()
        plt.savefig(f"{RESULTS_DIR}/graficos/confusao/matriz_confusao_{nome.replace(' ', '_').lower()}.png", dpi=300)
        plt.close()

        # Visualizar curva ROC
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
        plt.plot([0, 1], [0, 1], 'k--', lw=2)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('Taxa de Falsos Positivos', fontsize=12)
        plt.ylabel('Taxa de Verdadeiros Positivos', fontsize=12)
        plt.title(f'Curva ROC - {nome}', fontsize=15)
        plt.legend(loc="lower right")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f"{RESULTS_DIR}/graficos/roc/curva_roc_{nome.replace(' ', '_').lower()}.png", dpi=300)
        plt.close()

        # Para Random Forest, visualizar importância das features
        if nome == 'Random Forest':
            importances = modelo.feature_importances_
            indices = np.argsort(importances)[::-1]

            plt.figure(figsize=(12, 8))
            plt.bar(range(len(indices[:15])), importances[indices[:15]], align='center')
            plt.xticks(range(len(indices[:15])), [feature_names[i] for i in indices[:15]], rotation=90)
            plt.title('Importância das Features - Random Forest', fontsize=15)
            plt.tight_layout()
            plt.savefig(f"{RESULTS_DIR}/graficos/importancia/importancia_features_random_forest.png", dpi=300)
            plt.close()

    # Criar DataFrame com resultados
    resultados_df = pd.DataFrame(resultados)

    # Salvar resultados
    resultados_df.to_csv(f"{RESULTS_DIR}/resultados_modelos_classicos.csv", index=False)

    return resultados_df


# =====================================================================
# FUNÇÕES PARA TREINAMENTO E AVALIAÇÃO DE MODELOS
# =====================================================================

def treinar_modelo(modelo, x_train, y_train, x_val, y_val,
                   nome_modelo, batch_size=32, epochs=100,
                   class_weight=None, verbose=1):
    """
    Treina o modelo de rede neural.

    Args:
        modelo (tf.keras.Model): Modelo a ser treinado
        x_train (np.ndarray): Dados de treinos
        y_train (np.ndarray): Rótulos de treinos
        x_val (np.ndarray): Dados de validação
        y_val (np.ndarray): Rótulos de validação
        nome_modelo (str): Nome do modelo para salvar
        batch_size (int): Tamanho do batch
        epochs (int): Número máximo de épocas
        class_weight (dict): Pesos para as classes
        verbose (int): Nível de verbosidade

    Returns:
        tuple: (modelo treinado, histórico de treinos)
    """
    logger.info(f"Iniciando treinamento do modelo {nome_modelo}")
    logger.info(f"Batch size: {batch_size}, Épocas: {epochs}")

    # Verificar se é necessário reshape para CNN ou modelos híbridos
    if 'cnn' in nome_modelo.lower() or 'hibrido' in nome_modelo.lower():
        if len(x_train.shape) == 2:
            logger.info("Adicionando dimensão extra para dados CNN/híbridos")
            x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
            x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)

    # Calcular class weights se não fornecidos
    if class_weight is None and len(np.unique(y_train)) == 2:
        logger.info("Calculando pesos de classe para balanceamento")
        n_samples = len(y_train)
        n_positives = np.sum(y_train)
        n_negatives = n_samples - n_positives

        weight_for_0 = (1 / n_negatives) * (n_samples / 2.0)
        weight_for_1 = (1 / n_positives) * (n_samples / 2.0)

        class_weight = {0: weight_for_0, 1: weight_for_1}
        logger.info(f"Class weights: {class_weight}")

    # Criar callbacks
    callbacks = criar_callbacks(nome_modelo)

    # Registrar tempo de início
    start_time = time.time()

    # Treinar o modelo
    history = modelo.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        batch_size=batch_size,
        epochs=epochs,
        class_weight=class_weight,
        callbacks=callbacks,
        verbose=verbose
    )

    # Calcular tempo de treinamento
    training_time = time.time() - start_time
    logger.info(f"Treinamento concluído em {training_time:.2f} segundos")

    # Carregar o melhor modelo
    try:
        logger.info(f"Carregando o melhor modelo de {RESULTS_DIR}/modelos/{nome_modelo}_best.h5")
        modelo = load_model(f"{RESULTS_DIR}/modelos/{nome_modelo}_best.h5")
    except:
        logger.warning("Não foi possível carregar o melhor modelo. Usando o modelo atual.")

    # Salvar o modelo final
    modelo.save(f"{RESULTS_DIR}/modelos/{nome_modelo}_final.h5")
    logger.info(f"Modelo final salvo em {RESULTS_DIR}/modelos/{nome_modelo}_final.h5")

    # Salvar histórico de treino
    hist_df = pd.DataFrame(history.history)
    hist_df.to_csv(f"{RESULTS_DIR}/history/{nome_modelo}_history_full.csv", index=False)


def avaliar_modelo(modelo, x_test, y_test, nome_modelo, threshold=0.5):
    """
    Avalia o desempenho do modelo nos dados de teste.

    Args:
        modelo (tf.keras.Model): Modelo treinado
        x_test (np.ndarray): Dados de teste
        y_test (np.ndarray): Rótulos de teste
        nome_modelo (str): Nome do modelo para salvar resultados
        threshold (float): Limiar para classificação binária

    Returns:
        dict: Dicionário com métricas de desempenho
    """
    logger.info(f"Avaliando modelo {nome_modelo} nos dados de teste")

    # Verificar se é necessário reshape para CNN ou modelos híbridos
    if 'cnn' in nome_modelo.lower() or 'hibrido' in nome_modelo.lower():
        if len(x_test.shape) == 2:
            logger.info("Adicionando dimensão extra para dados CNN/híbridos")
            x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

    # Fazer predições
    y_pred_prob = modelo.predict(x_test)
    y_pred = (y_pred_prob > threshold).astype(int).flatten()
    y_pred_prob = y_pred_prob.flatten()

    # Calcular métricas
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_pred_prob),
        'average_precision': average_precision_score(y_test, y_pred_prob)
    }

    # Exibir métricas
    logger.info("Métricas de desempenho:")
    for metric, value in metrics.items():
        logger.info(f"{metric}: {value:.4f}")

    # Salvar métricas em CSV
    pd.DataFrame([metrics]).to_csv(f"{RESULTS_DIR}/{nome_modelo}_metricas.csv", index=False)

    # Matriz de confusão
    cm = confusion_matrix(y_test, y_pred)

    # Relatório de classificação
    report = classification_report(y_test, y_pred, output_dict=True)
    pd.DataFrame(report).transpose().to_csv(f"{RESULTS_DIR}/{nome_modelo}_classification_report.csv")

    # Visualizações
    visualizar_resultados(y_test, y_pred, y_pred_prob, nome_modelo)

    return metrics, y_pred, y_pred_prob


def visualizar_resultados(y_true, y_pred, y_prob, nome_modelo):
    """
    Cria visualizações para os resultados do modelo.

    Args:
        y_true (np.ndarray): Rótulos verdadeiros
        y_pred (np.ndarray): Predições binárias
        y_prob (np.ndarray): Probabilidades preditas
        nome_modelo (str): Nome do modelo para salvar visualizações
    """
    logger.info(f"Gerando visualizações para resultados do modelo {nome_modelo}")

    # 1. Matriz de confusão
    plt.figure(figsize=(10, 8))
    cm = confusion_matrix(y_true, y_pred)

    # Normalizar matriz de confusão
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    # Plotar matriz de confusão
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Matriz de Confusão - {nome_modelo}', fontsize=15)
    plt.ylabel('Valor Real', fontsize=12)
    plt.xlabel('Valor Predito', fontsize=12)
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/graficos/{nome_modelo}_matriz_confusao.png", dpi=300)
    plt.close()

    # Matriz de confusão normalizada
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues', cbar=False)
    plt.title(f'Matriz de Confusão Normalizada - {nome_modelo}', fontsize=15)
    plt.ylabel('Valor Real', fontsize=12)
    plt.xlabel('Valor Predito', fontsize=12)
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/graficos/{nome_modelo}_matriz_confusao_norm.png", dpi=300)
    plt.close()

    # 2. Curva ROC
    plt.figure(figsize=(10, 8))
    fpr, tpr, thresholds = roc_curve(y_true, y_prob)
    roc_auc = auc(fpr, tpr)

    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taxa de Falsos Positivos', fontsize=12)
    plt.ylabel('Taxa de Verdadeiros Positivos', fontsize=12)
    plt.title(f'Curva ROC - {nome_modelo}', fontsize=15)
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/graficos/{nome_modelo}_curva_roc.png", dpi=300)
    plt.close()

    # 3. Curva Precision-Recall
    plt.figure(figsize=(10, 8))
    precision, recall, _ = precision_recall_curve(y_true, y_prob)
    avg_precision = average_precision_score(y_true, y_prob)

    plt.plot(recall, precision, color='blue', lw=2, label=f'PR curve (AP = {avg_precision:.4f})')
    plt.axhline(y=sum(y_true) / len(y_true), color='red', linestyle='--', label='Baseline')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall', fontsize=12)
    plt.ylabel('Precision', fontsize=12)
    plt.title(f'Curva Precision-Recall - {nome_modelo}', fontsize=15)
    plt.legend(loc="lower left")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/graficos/{nome_modelo}_curva_precision_recall.png", dpi=300)
    plt.close()

    # 4. Histograma de probabilidades
    plt.figure(figsize=(12, 8))

    # Separar probabilidades por classe
    prob_pos = y_prob[y_true == 1]
    prob_neg = y_prob[y_true == 0]

    plt.hist(prob_pos, bins=20, alpha=0.5, color='green', label='Classe Positiva (Diabetes)')
    plt.hist(prob_neg, bins=20, alpha=0.5, color='red', label='Classe Negativa (Não Diabetes)')

    plt.axvline(x=0.5, color='black', linestyle='--', label='Limiar (0.5)')
    plt.xlabel('Probabilidade Predita', fontsize=12)
    plt.ylabel('Contagem', fontsize=12)
    plt.title(f'Distribuição de Probabilidades - {nome_modelo}', fontsize=15)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/graficos/{nome_modelo}_distribuicao_probabilidades.png", dpi=300)
    plt.close()


def visualizar_historico_treinamento(historico, nome_modelo):
    """
    Visualiza o histórico de treinamento do modelo.

    Args:
        historico (tf.keras.callbacks.History): Histórico de treinamento
        nome_modelo (str): Nome do modelo para salvar visualizações
    """
    logger.info(f"Visualizando histórico de treinamento do modelo {nome_modelo}")

    # 1. Curvas de perda (loss)
    plt.figure(figsize=(12, 8))
    plt.plot(historico['loss'], label='Treino', color='blue')
    plt.plot(historico['val_loss'], label='Validação', color='orange')
    plt.title(f'Curvas de Perda - {nome_modelo}', fontsize=15)
    plt.xlabel('Época', fontsize=12)
    plt.ylabel('Perda (Binary Crossentropy)', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/graficos/{nome_modelo}_curvas_perda.png", dpi=300)
    plt.close()

    # 2. Curvas de acurácia
    plt.figure(figsize=(12, 8))
    plt.plot(historico['accuracy'], label='Treino', color='blue')
    plt.plot(historico['val_accuracy'], label='Validação', color='orange')
    plt.title(f'Curvas de Acurácia - {nome_modelo}', fontsize=15)
    plt.xlabel('Época', fontsize=12)
    plt.ylabel('Acurácia', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/graficos/{nome_modelo}_curvas_acuracia.png", dpi=300)
    plt.close()

    # 3. Curvas de métricas adicionais
    if 'precision' in historico.columns and 'recall' in historico.columns and 'auc' in historico.columns:
        plt.figure(figsize=(18, 6))

        plt.subplot(1, 3, 1)
        plt.plot(historico['precision'], label='Treino', color='blue')
        plt.plot(historico['val_precision'], label='Validação', color='orange')
        plt.title('Precisão', fontsize=13)
        plt.xlabel('Época', fontsize=11)
        plt.ylabel('Precisão', fontsize=11)
        plt.legend()
        plt.grid(True, alpha=0.3)

        plt.subplot(1, 3, 2)
        plt.plot(historico['recall'], label='Treino', color='blue')
        plt.plot(historico['val_recall'], label='Validação', color='orange')
        plt.title('Recall', fontsize=13)
        plt.xlabel('Época', fontsize=11)
        plt.ylabel('Recall', fontsize=11)
        plt.legend()
        plt.grid(True, alpha=0.3)

        plt.subplot(1, 3, 3)
        plt.plot(historico['auc'], label='Treino', color='blue')
        plt.plot(historico['val_auc'], label='Validação', color='orange')
        plt.title('AUC', fontsize=13)
        plt.xlabel('Época', fontsize=11)
        plt.ylabel('AUC', fontsize=11)
        plt.legend()
        plt.grid(True, alpha=0.3)

        plt.suptitle(f'Métricas de Treinamento - {nome_modelo}', fontsize=16, y=1.05)
        plt.tight_layout()
        plt.savefig(f"{RESULTS_DIR}/graficos/{nome_modelo}_curvas_metricas.png", dpi=300)
        plt.close()

    # 4. Curvas de aprendizado (Learning Curves)
    plt.figure(figsize=(15, 10))

    # Subplot para loss
    plt.subplot(2, 2, 1)
    plt.plot(historico['loss'], label='Treino', color='blue')
    plt.plot(historico['val_loss'], label='Validação', color='orange')
    plt.title('Perda (Loss)', fontsize=13)
    plt.xlabel('Época', fontsize=11)
    plt.ylabel('Perda', fontsize=11)
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Subplot para accuracy
    plt.subplot(2, 2, 2)
    plt.plot(historico['accuracy'], label='Treino', color='blue')
    plt.plot(historico['val_accuracy'], label='Validação', color='orange')
    plt.title('Acurácia', fontsize=13)
    plt.xlabel('Época', fontsize=11)
    plt.ylabel('Acurácia', fontsize=11)
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Calcular diferença entre treino e validação
    if 'loss' in historico.columns and 'val_loss' in historico.columns:
        plt.subplot(2, 2, 3)
        plt.plot(historico['loss'] - historico['val_loss'], label='Diferença', color='green')
        plt.axhline(y=0, color='red', linestyle='--')
        plt.title('Diferença de Perda (Treino - Validação)', fontsize=13)
        plt.xlabel('Época', fontsize=11)
        plt.ylabel('Diferença', fontsize=11)
        plt.legend()
        plt.grid(True, alpha=0.3)

    # Calcular razão entre treino e validação
    if 'accuracy' in historico.columns and 'val_accuracy' in historico.columns:
        plt.subplot(2, 2, 4)
        plt.plot(historico['accuracy'] / historico['val_accuracy'], label='Razão', color='purple')
        plt.axhline(y=1, color='red', linestyle='--')
        plt.title('Razão de Acurácia (Treino / Validação)', fontsize=13)
        plt.xlabel('Época', fontsize=11)
        plt.ylabel('Razão', fontsize=11)
        plt.legend()
        plt.grid(True, alpha=0.3)

    plt.suptitle(f'Curvas de Aprendizado - {nome_modelo}', fontsize=16, y=1.05)
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/graficos/{nome_modelo}_curvas_aprendizado.png", dpi=300)
    plt.close()

In [ ]:
# =====================================================================
# FUNÇÕES PARA INTERPRETABILIDADE DE MODELOS
# =====================================================================

def analisar_importancia_variaveis(modelo, x_test, y_test, feature_names, nome_modelo, n_repeats=10, random_state=42):
    """
    Analisa a importância das variáveis no modelo usando cálculo manual de permutation importance com roc_auc.

    Args:
        modelo (tf.keras.Model): Modelo treinado
        x_test (np.ndarray): Dados de teste
        y_test (np.ndarray): Rótulos de teste
        feature_names (list): Nomes das features
        nome_modelo (str): Nome do modelo para salvar resultados
        n_repeats (int): Número de repetições para permutação
        random_state (int): Semente aleatória

    Returns:
        pd.DataFrame: DataFrame com importância das variáveis
    """
    logger.info(f"Analisando importância das variáveis para o modelo {nome_modelo} (cálculo manual)")

    # Determina se o modelo é CNN ou híbrido com base no nome
    is_cnn_or_hybrid = 'cnn' in nome_modelo.lower() or 'hibrido' in nome_modelo.lower()

    # Garante que x_test seja 2D para a lógica de permutação
    x_test_2d = x_test
    if is_cnn_or_hybrid and len(x_test.shape) == 3:
        logger.info("Convertendo dados 3D para 2D para análise de importância")
        x_test_2d = x_test.reshape(x_test.shape[0], x_test.shape[1])

    # Função para obter probabilidades do modelo Keras (considerando reshape)
    def get_proba(x_input):
        x_input_internal = x_input  # Usa cópia para evitar modificar original
        if is_cnn_or_hybrid and len(x_input_internal.shape) == 2:
            x_input_internal = x_input_internal.reshape(x_input_internal.shape[0], x_input_internal.shape[1], 1)
        return modelo.predict(x_input_internal)[:, 0]  # Retorna prob da classe 1

    # Calcula a pontuação base (AUC)
    try:
        baseline_proba = get_proba(x_test_2d)
        baseline_score = roc_auc_score(y_test, baseline_proba)
        logger.info(f"Baseline ROC AUC score: {baseline_score:.4f}")
    except Exception as e:
        logger.error(f"Erro ao calcular baseline score para {nome_modelo}: {e}")
        return pd.DataFrame()  # Retorna DataFrame vazio em caso de falha

    # Inicializa arrays para armazenar importâncias
    importances_mean = np.zeros(x_test_2d.shape[1])
    importances_std = np.zeros(x_test_2d.shape[1])
    all_perm_scores = [[] for _ in range(x_test_2d.shape[1])]

    # Itera sobre cada feature
    logger.info("Calculando importância por permutação manual (pode levar algum tempo)")
    rng = np.random.RandomState(random_state)
    for col_idx in tqdm(range(x_test_2d.shape[1]), desc=f"Permutando features - {nome_modelo}"):
        original_col = x_test_2d[:, col_idx].copy()
        perm_scores = []

        for _ in range(n_repeats):
            # Permuta a coluna atual
            x_test_permuted = x_test_2d.copy()
            x_test_permuted[:, col_idx] = rng.permutation(original_col)

            # Calcula a pontuação com a coluna permutada
            try:
                permuted_proba = get_proba(x_test_permuted)
                perm_score = roc_auc_score(y_test, permuted_proba)
                perm_scores.append(perm_score)
                all_perm_scores[col_idx].append(perm_score)
            except Exception as e:
                logger.error(f"Erro ao calcular score para feature {col_idx} permutada: {e}")
                perm_scores.append(np.nan)  # Adiciona NaN em caso de erro
                all_perm_scores[col_idx].append(np.nan)

        # Calcula a importância como a diferença para a baseline
        valid_perm_scores = [s for s in perm_scores if not np.isnan(s)]
        if valid_perm_scores:
            importances_mean[col_idx] = baseline_score - np.mean(valid_perm_scores)
            importances_std[col_idx] = np.std(valid_perm_scores)
        else:
            importances_mean[col_idx] = 0  # Define importância como 0 se todos os cálculos falharam
            importances_std[col_idx] = 0

    # Organizar resultados
    importancia = pd.DataFrame({
        'Feature': feature_names[:x_test_2d.shape[1]],  # Garantir que o número de features corresponda
        'Importância': importances_mean,
        'Desvio Padrão': importances_std
    })

    importancia = importancia.sort_values('Importância', ascending=False)

    # Salvar resultados
    importancia.to_csv(f"{RESULTS_DIR}/{nome_modelo}_importancia_variaveis_manual.csv", index=False)

    # Visualizar importância das variáveis (top 15)
    plt.figure(figsize=(12, 10))
    top_features = importancia.head(15)
    sns.barplot(x='Importância', y='Feature', data=top_features, palette='viridis', hue='Feature', legend=False)
    plt.title(f'Top 15 Variáveis Mais Importantes (Manual) - {nome_modelo}', fontsize=15)
    plt.xlabel('Importância (Queda no ROC AUC)', fontsize=12)
    plt.ylabel('Variável', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/graficos/{nome_modelo}_importancia_variaveis_manual.png", dpi=300)
    plt.close()

    # Visualizar importância com barras de erro
    plt.figure(figsize=(12, 10))
    top_features = importancia.head(10)
    plt.errorbar(
        x=top_features['Importância'],
        y=range(len(top_features)),
        xerr=top_features['Desvio Padrão'],  # Nota: std aqui é do score permutado, não da importância
        fmt='o',
        capsize=5,
        color='blue'
    )
    plt.yticks(range(len(top_features)), top_features['Feature'])
    plt.title(f'Top 10 Variáveis com Desvio Padrão (Manual) - {nome_modelo}', fontsize=15)
    plt.xlabel('Importância (Queda no ROC AUC)', fontsize=12)
    plt.ylabel('Variável', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/graficos/{nome_modelo}_importancia_variaveis_erro_manual.png", dpi=300)
    plt.close()

    return importancia


def analisar_shap_values(modelo, x_test, feature_names, nome_modelo, max_display=10):
    """
    Analisa a importância das variáveis usando SHAP values.

    Args:
        modelo (tf.keras.Model): Modelo treinado
        x_test (np.ndarray): Dados de teste
        feature_names (list): Nomes das features
        nome_modelo (str): Nome do modelo para salvar resultados
        max_display (int): Número máximo de features para exibir

    Returns:
        tuple: (explainer, shap_values)
    """
    logger.info(f"Analisando SHAP values para o modelo {nome_modelo}")

    # Verificar se é necessário reshape para CNN ou modelos híbridos
    x_test_2d = x_test
    if 'cnn' in nome_modelo.lower() or 'hibrido' in nome_modelo.lower():
        if len(x_test.shape) == 3:
            logger.info("Convertendo dados 3D para 2D para análise SHAP")
            x_test_2d = x_test.reshape(x_test.shape[0], x_test.shape[1])


    try:
        # Usar uma amostra dos dados de teste para eficiência
        sample_size = min(100, x_test_2d.shape[0])
        x_sample = x_test_2d[:sample_size]

        # Garantir que x_sample está 2D para o SHAP
        if len(x_sample.shape) == 3 and x_sample.shape[2] == 1:
            logger.info("Convertendo dados 3D para 2D para análise SHAP")
            x_sample_2d = x_sample.reshape(x_sample.shape[0], x_sample.shape[1])
        else:
            x_sample_2d = x_sample

        # Calcular SHAP values
        logger.info("Calculando SHAP values (pode levar algum tempo)")
        explainer = shap.Explainer(modelo, x_sample_2d)
        shap_values = explainer.shap_values(x_sample_2d)

        # Resumo das contribuições das features
        plt.figure(figsize=(12, 10))
        shap.summary_plot(
            shap_values,
            x_sample_2d,
            feature_names=feature_names[:x_test_2d.shape[1]],
            max_display=max_display,
            show=False
        )
        plt.title(f'SHAP Summary Plot - {nome_modelo}', fontsize=15)
        plt.tight_layout()
        plt.savefig(f"{RESULTS_DIR}/graficos/shap/{nome_modelo}_shap_summary.png", dpi=300)
        plt.close()

        # Gráfico de barras com importância média absoluta
        plt.figure(figsize=(12, 10))
        shap.summary_plot(
            shap_values,
            x_sample_2d,
            feature_names=feature_names[:x_test_2d.shape[1]],
            plot_type='bar',
            max_display=max_display,
            show=False
        )
        plt.title(f'SHAP Feature Importance - {nome_modelo}', fontsize=15)
        plt.tight_layout()
        plt.savefig(f"{RESULTS_DIR}/graficos/shap/{nome_modelo}_shap_importance.png", dpi=300)
        plt.close()

        # Gráficos de dependência para as top 3 features
        shap_df = pd.DataFrame(shap_values, columns=feature_names[:x_test_2d.shape[1]])
        mean_abs_shap = np.abs(shap_df).mean().sort_values(ascending=False)
        top_features = mean_abs_shap.index[:3].tolist()

        for feature in top_features:
            plt.figure(figsize=(10, 8))
            feature_idx = list(feature_names[:x_test_2d.shape[1]]).index(feature)
            shap.dependence_plot(
                feature_idx,
                shap_values,
                x_sample_2d,
                feature_names=feature_names[:x_test_2d.shape[1]],
                show=False
            )
            plt.title(f'SHAP Dependence Plot - {feature} - {nome_modelo}', fontsize=15)
            plt.tight_layout()
            plt.savefig(f"{RESULTS_DIR}/graficos/shap/{nome_modelo}_shap_dependence_{feature}.png", dpi=300)
            plt.close()

        # Gráfico de força para algumas amostras individuais
        for i in range(min(3, x_sample_2d.shape[0])):
            plt.figure(figsize=(16, 6))
            shap.force_plot(
                explainer.expected_value,
                shap_values[i],
                x_sample_2d[i],
                feature_names=feature_names[:x_test_2d.shape[1]],
                matplotlib=True,
                show=False
            )
            plt.title(f'SHAP Force Plot - Amostra {i + 1} - {nome_modelo}', fontsize=15)
            plt.tight_layout()
            plt.savefig(f"{RESULTS_DIR}/graficos/shap/{nome_modelo}_shap_force_plot_sample_{i + 1}.png", dpi=300)
            plt.close()

        return explainer, shap_values

    except Exception as e:
        logger.error(f"Erro ao calcular SHAP values: {e}")
        logger.info("Continuando com outras análises...")
        return None, None


def analisar_lime(modelo, x_train, x_test, y_test, feature_names, nome_modelo, num_amostras=3):
    """
    Analisa o modelo usando LIME (Local Interpretable Model-agnostic Explanations).

    Args:
        modelo (tf.keras.Model): Modelo treinado
        x_train (np.ndarray): Dados de treinamento
        x_test (np.ndarray): Dados de teste
        y_test (np.ndarray): Rótulos de teste
        feature_names (list): Nomes das features
        nome_modelo (str): Nome do modelo para salvar resultados
        num_amostras (int): Número de amostras para explicar

    Returns:
        lime.lime_tabular.LimeTabularExplainer: Explainer LIME
    """
    logger.info(f"Analisando modelo {nome_modelo} usando LIME")

    # Verificar se é necessário reshape para CNN ou modelos híbridos
    x_train_2d = x_train
    x_test_2d = x_test
    if 'cnn' in nome_modelo.lower() or 'hibrido' in nome_modelo.lower():
        if len(x_train.shape) == 3:
            logger.info("Convertendo dados 3D para 2D para análise LIME")
            x_train_2d = x_train.reshape(x_train.shape[0], x_train.shape[1])
            x_test_2d = x_test.reshape(x_test.shape[0], x_test.shape[1])

    # Função de predição para LIME
    def predict_func(x):
        # Garantir que X seja um array numpy e tenha o formato correto
        if not isinstance(x, np.ndarray):
            x = np.array(x)

        # Garantir que X seja 2D
        if len(x.shape) == 1:
            x = x.reshape(1, -1)

        # Garantir que X tenha o número correto de features
        if x.shape[1] != x_test_2d.shape[1]:
            logger.warning(f"Shape mismatch: esperado {x_test_2d.shape[1]} features, recebido {x.shape[1]}")
            return np.array([[0.5, 0.5]] * x.shape[0])

        try:
            if 'cnn' in nome_modelo.lower() or 'hibrido' in nome_modelo.lower():
                x_reshaped = x.reshape(x.shape[0], x.shape[1], 1)
                proba = modelo.predict(x_reshaped, verbose=0)
            else:
                proba = modelo.predict(x, verbose=0)

            # Garante que proba é 1D para a classe positiva
            proba_pos = proba.reshape(-1)

            # Clamp probabilidades para evitar valores extremos
            proba_pos = np.clip(proba_pos, 1e-7, 1 - 1e-7)

            # Retorna as probabilidades para as duas classes
            proba_neg = 1 - proba_pos
            return np.column_stack([proba_neg, proba_pos])

        except Exception as e:
            logger.warning(f"Erro na predição LIME: {e}. Retornando probabilidades padrão.")
            # Retornar probabilidades padrão em caso de erro
            return np.array([[0.5, 0.5]] * x.shape[0])

    try:
        # Criar explainer LIME
        logger.info("Criando LIME explainer")
        explainer = lime_tabular.LimeTabularExplainer(
            x_train_2d,
            feature_names=feature_names[:x_test_2d.shape[1]],
            class_names=['Não Diabetes', 'Diabetes'],
            mode='classification'
        )

        # Selecionar amostras para explicar
        amostras_para_explicar = np.random.choice(x_test.shape[0], size=num_amostras, replace=False)

        for i in amostras_para_explicar:
            logger.info(f"Explicando amostra {i}")

            # Gerar explicação
            exp = explainer.explain_instance(
                x_test_2d[i],
                predict_func,
                num_features=10
            )

            # Visualizar explicação
            plt.figure(figsize=(12, 8))
            exp.as_pyplot_figure()
            plt.title(f'Explicação LIME - Amostra {i} - {nome_modelo}', fontsize=15)
            plt.tight_layout()
            plt.savefig(f"{RESULTS_DIR}/graficos/lime/{nome_modelo}_lime_amostra_{i}.png", dpi=300)
            plt.close()

        return explainer

    except Exception as e:
        logger.error(f"Erro ao gerar explicações LIME: {e}")
        logger.info("Continuando com outras análises...")
        return None


In [ ]:
# 1. Carregar e analisar dados
df = carregar_dados("diabetes_prediction_dataset.csv")
analisar_dados(df)

# 2. Pré-processamento dos dados
x_train, X_val, x_test, y_train, y_val, y_test, pre_processador, feature_names = pre_processar_dados(
    df,
    metodo_normalizacao='standard',
    metodo_balanceamento='smote',
    tratar_outliers=True
)

# Reshape para CNN
x_train_cnn = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
X_val_cnn = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_test_cnn = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)


In [ ]:
# 3. Treinar modelos clássicos
resultados_classicos = treinar_modelos_classicos(x_train, y_train, x_test, y_test, feature_names)

In [ ]:
# 4. Treinar e avaliar modelo MLP
logger.info("Treinando modelo MLP")
modelo_mlp = criar_modelo_mlp(
    input_shape=(x_train.shape[1],),
    camadas=[128, 64, 32],
    activations=['relu', 'relu', 'relu'],
    dropout_rates=[0.3, 0.3, 0.3],
    regularization=0.001,
    learning_rate=0.001,
    batch_norm=True
)

treinar_modelo(
    modelo_mlp, x_train, y_train, X_val, y_val,
    nome_modelo="mlp",
    batch_size=128,
    epochs=100
)

In [ ]:
# 5. Treinar e avaliar modelo CNN
logger.info("Treinando modelo CNN")
modelo_cnn = criar_modelo_cnn(
    input_shape=(x_train.shape[1], 1),
    filtros=[64, 32, 16],
    kernel_sizes=[3, 3, 3],
    ativacoes=['relu', 'relu', 'relu'],
    dropout_rates=[0.3, 0.3, 0.3],
    pool_sizes=[2, 2, 2],
    regularizacao=0.001,
    learning_rate=0.005,
    batch_norm=True
)

treinar_modelo(
    modelo_cnn, x_train_cnn, y_train, X_val_cnn, y_val,
    nome_modelo="cnn",
    batch_size=128,
    epochs=100,
)

In [ ]:
# 6. Treinar e avaliar modelo híbrido
logger.info("Treinando modelo híbrido CNN-LSTM")
modelo_hibrido = criar_modelo_hibrido(
    input_shape=(x_train.shape[1], 1),
    filtros_cnn=[64, 32, 16],
    kernel_sizes=[3, 3, 3],
    unidades_lstm=32,
    unidades_densas=[64, 32, 16],
    ativacoes=['relu', 'relu', 'relu'],
    dropout_rates=[0.3, 0.3, 0.3],
    regularizacao=0.001,
    learning_rate=0.001,
    batch_norm=True
)

treinar_modelo(
    modelo_hibrido, x_train_cnn, y_train, X_val_cnn, y_val,
    nome_modelo="hibrido",
    batch_size=128,
    epochs=100
)

In [ ]:
# Caminho do arquivo onde o modelo foi salvo
caminho_modelo_mlp = f"{RESULTS_DIR}/modelos/mlp_final.h5"
caminho_history_mlp = f"{RESULTS_DIR}/history/mlp_history.csv"
caminho_modelo_cnn = f"{RESULTS_DIR}/modelos/cnn_final.h5"
caminho_history_cnn = f"{RESULTS_DIR}/history/cnn_history.csv"
caminho_modelo_hibrido = f"{RESULTS_DIR}/modelos/hibrido_final.h5"
caminho_history_hibrido = f"{RESULTS_DIR}/history/hibrido_history.csv"

# Carregar o modelo híbrido
modelo_hibrido_treinado = load_model(caminho_modelo_hibrido)
historico_hibrido = pd.read_csv(caminho_history_hibrido)

# Carregar o modelo MLP
modelo_mlp_treinado = load_model(caminho_modelo_mlp)
historico_mlp = pd.read_csv(caminho_history_cnn)

# Carregar o modelo CNN
modelo_cnn_treinado = load_model(caminho_modelo_cnn)
historico_cnn = pd.read_csv(caminho_history_mlp)

In [ ]:
# Avaliar modelos
metricas_mlp, y_pred_mlp, y_prob_mlp = avaliar_modelo(
    modelo_mlp_treinado, x_test, y_test, nome_modelo="mlp"
)

visualizar_historico_treinamento(historico_mlp, "mlp")

metricas_hibrido, y_pred_hibrido, y_prob_hibrido = avaliar_modelo(
    modelo_hibrido_treinado, X_test_cnn, y_test, nome_modelo="hibrido"
)

visualizar_historico_treinamento(historico_hibrido, "hibrido")

metricas_cnn, y_pred_cnn, y_prob_cnn = avaliar_modelo(
    modelo_cnn_treinado, X_test_cnn, y_test, nome_modelo="cnn"
)

visualizar_historico_treinamento(historico_cnn, "cnn")

In [ ]:
# 7. Análise de importância de variáveis

importancia_mlp = analisar_importancia_variaveis(
    modelo_mlp_treinado, x_test, y_test, feature_names, "mlp"
)

importancia_cnn = analisar_importancia_variaveis(
    modelo_cnn_treinado, X_test_cnn, y_test, feature_names, "cnn"
)

importancia_hibrido = analisar_importancia_variaveis(
    modelo_hibrido_treinado, X_test_cnn, y_test, feature_names, "hibrido"
)

In [ ]:
# 8. Análise SHAP
explainer_mlp, shap_values_mlp = analisar_shap_values(
    modelo_mlp_treinado, x_test, feature_names, "mlp"
)

explainer_cnn, shap_values_cnn = analisar_shap_values(
    modelo_cnn_treinado, X_test_cnn, feature_names, "cnn"
)

explainer_cnn, shap_values_cnn = analisar_shap_values(
    modelo_cnn_treinado, X_test_cnn, feature_names, "hibrido"
)

In [ ]:
# 9. Análise LIME
lime_explainer_mlp = analisar_lime(
    modelo_mlp_treinado, x_train, x_test, y_test, feature_names, "mlp"
)

lime_explainer_cnn = analisar_lime(
    modelo_cnn_treinado, x_train_cnn, X_test_cnn, y_test, feature_names, "cnn"
)

lime_explainer_hibrido = analisar_lime(
    modelo_hibrido_treinado, x_train_cnn, X_test_cnn, y_test, feature_names, "hibrido"
)


In [ ]:
# Corrigir carregamento dos resultados dos modelos clássicos
try:
    # Tentar carregar o arquivo correto de resultados clássicos
    if os.path.exists(f"{RESULTS_DIR}/resultados_modelos_classicos.csv"):
        resultados_classicos = pd.read_csv(f"{RESULTS_DIR}/resultados_modelos_classicos.csv")
        logger.info(f"Resultados clássicos carregados: {len(resultados_classicos)} modelos")
    else:
        logger.error("Arquivo de resultados clássicos não encontrado")
        # Criar DataFrame vazio com estrutura correta
        resultados_classicos = pd.DataFrame(columns=['modelo', 'accuracy', 'balanced_accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'average_precision'])
except Exception as e:
    logger.error(f"Erro ao carregar resultados clássicos: {e}")
    resultados_classicos = pd.DataFrame(columns=['modelo', 'accuracy', 'balanced_accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'average_precision'])

# 10. Comparação de todos os modelos - com validação de NaN
try:
    # Verificar se as métricas dos modelos estão disponíveis e válidas
    modelos_validos = []

    # Validar MLP
    if 'metricas_mlp' in locals() and all(key in metricas_mlp for key in ['accuracy', 'balanced_accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'average_precision']):
        if not any(pd.isna(list(metricas_mlp.values()))):
            modelos_validos.append({
                'modelo': 'MLP',
                'accuracy': metricas_mlp['accuracy'],
                'balanced_accuracy': metricas_mlp['balanced_accuracy'],
                'precision': metricas_mlp['precision'],
                'recall': metricas_mlp['recall'],
                'f1': metricas_mlp['f1'],
                'roc_auc': metricas_mlp['roc_auc'],
                'average_precision': metricas_mlp['average_precision']
            })
            logger.info("MLP: métricas válidas adicionadas")
        else:
            logger.warning("MLP: métricas contêm valores NaN")
    else:
        logger.warning("MLP: métricas não disponíveis ou incompletas")

    # Validar CNN
    if 'metricas_cnn' in locals() and all(key in metricas_cnn for key in ['accuracy', 'balanced_accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'average_precision']):
        if not any(pd.isna(list(metricas_cnn.values()))):
            modelos_validos.append({
                'modelo': 'CNN',
                'accuracy': metricas_cnn['accuracy'],
                'balanced_accuracy': metricas_cnn['balanced_accuracy'],
                'precision': metricas_cnn['precision'],
                'recall': metricas_cnn['recall'],
                'f1': metricas_cnn['f1'],
                'roc_auc': metricas_cnn['roc_auc'],
                'average_precision': metricas_cnn['average_precision']
            })
            logger.info("CNN: métricas válidas adicionadas")
        else:
            logger.warning("CNN: métricas contêm valores NaN")
    else:
        logger.warning("CNN: métricas não disponíveis ou incompletas")

    # Validar Híbrido
    if 'metricas_hibrido' in locals() and all(key in metricas_hibrido for key in ['accuracy', 'balanced_accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'average_precision']):
        if not any(pd.isna(list(metricas_hibrido.values()))):
            modelos_validos.append({
                'modelo': 'Híbrido CNN-LSTM',
                'accuracy': metricas_hibrido['accuracy'],
                'balanced_accuracy': metricas_hibrido['balanced_accuracy'],
                'precision': metricas_hibrido['precision'],
                'recall': metricas_hibrido['recall'],
                'f1': metricas_hibrido['f1'],
                'roc_auc': metricas_hibrido['roc_auc'],
                'average_precision': metricas_hibrido['average_precision']
            })
            logger.info("Híbrido: métricas válidas adicionadas")
        else:
            logger.warning("Híbrido: métricas contêm valores NaN")
    else:
        logger.warning("Híbrido: métricas não disponíveis ou incompletas")

    # Criar DataFrame com modelos válidos
    if modelos_validos:
        resultados_redes = pd.DataFrame(modelos_validos)
        logger.info(f"Criado DataFrame com {len(resultados_redes)} modelos de redes neurais válidos")
    else:
        logger.warning("Nenhum modelo de rede neural válido encontrado")
        resultados_redes = pd.DataFrame(columns=['modelo', 'accuracy', 'balanced_accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'average_precision'])

    # Executar comparação apenas se houver dados válidos
    if len(resultados_classicos) > 0 or len(resultados_redes) > 0:
        comparar_todos_modelos(resultados_classicos, resultados_redes)
        logger.info("Comparação de modelos executada com sucesso")
    else:
        logger.error("Nenhum resultado válido disponível para comparação")

except Exception as e:
    logger.error(f"Erro na comparação de modelos: {e}")
    import traceback
    traceback.print_exc()
